In diesem Notebook soll das Auf- und Abschwingen eines Root-Raised-Cosine (rrc) Pulses  akustische hörbar gemacht werden.
Zuerst wird der rrc-Puls als Funktion definiert, danach wird eine einzelner Puls erzeugt, mit einer Trägerfrequenz gemischt und anschließen als Audiodatei gespeichert.

In [ ]:
#Import all python dependencies we need!
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write, read

#Define variables
T = 0.2                         # symbol duration in s
f_t =500                        # carrier frequency
rate = np.int32(44100)          # rate at which audio signal is sampled
n_up = np.int32(rate*T)         # oversampling factor -> samples per symbol
beta= 0.8                       # 0 <= beta <= 1;  Roll-off factor of rrc, 0 -> sinc(t)
K = 10                           # Length of rrc in symbols, must be even number


Im Folgenden definieren wir den rrc als aufrufbare Funktion:

In [ ]:
def get_rrc(K, n_up, T, beta):
    
    ''' 
    Determines coefficients of an RC filter 
    
    Formula out of: John B. Anderson: Digital transmission engineering.
    
    NOTE: Length of the IR has to be an odd number
    
    IN: length of IR in symbols, upsampling factor per symbol, symbol time, roll-off factor
    OUT: filter coefficients
    '''
    N = K*n_up+1                        #number of samples for filter
    T_delta = T/n_up                    #time resolution of upsampled filter
    sample_num = np.arange(N)    
    h_rrc = np.zeros(N, dtype=float)

    for x in sample_num:
        t = np.array([(x-N/2)*T_delta])
        if t == 0.0:
            scaling = 1/np.sqrt(T)
            equation = 1-beta+(4*beta/np.pi)
            h_rrc[x] = scaling * equation
        elif beta != 0 and t == (T/(4*beta) or -T/(4*beta)):
            scaling = beta/np.sqrt(2*T)
            equation = (1+(2/np.pi))*np.sin(np.pi/(4*beta)) + (1-2/np.pi)*np.cos(np.pi/(4*beta))
            h_rrc[x] = scaling * equation
        else:
            scaling = 1/np.sqrt(T)
            numerator = np.sin(np.pi*(1-beta)*t/T) + (4*beta*t/T)*np.cos(np.pi*(1+beta)*t/T)
            denominator = (np.pi*t/T)*(1-np.square(4*beta*t/T))
            equation = numerator / denominator
            h_rrc[x] = scaling * equation
            if denominator == 0:            #Dirty workarround for denom=0, negligible if n_up is sufficiently big
                h_rrc[x] = h_rrc[x-1]           
    return h_rrc 

Nun können wir uns über die Funktion die Koeffizienten des Pulses $g(t)$ eines rrc-Pulses mit den von uns oben spezifizierten Parametern ausgeben lassen.
Um die Impulsantwort $g(t)$ des Pulses "hörbar" zu machen erzeugen wir durch $g_\mathrm{BP}(t)=\text{Re}\left\{g(t) \mathrm{e}^{\mathrm{j} 2 \pi f_\mathrm{T} t} \right\}$ ein Bandpasspuls $g_\mathrm{BP}(t)$ mit Mittenfrequenz $f_\mathrm{T}$, wobei die komplexe Einhüllende von $g_\mathrm{BP}(t)$ der Impulsantwort $g(t)$ des rrc-Pulses entspricht.

In [ ]:
#get impulse response of filter
g = get_rrc(K,n_up,T,beta)

#Do passband conversion
t = np.linspace(-K/2,K/2,np.size(g))*T
g_bp = np.real(g*np.exp(1j*2*np.pi*f_t*t))

#Plot g(t) and g_BP(t)
plt.plot(t,g_bp,color='orange',label='$g_\mathrm{BP}(t)$',linewidth=0.3)
plt.plot(t,g,color='k',label='$g(t)$')
plt.grid(True)
plt.legend()
plt.xlabel('$t$ [s]')
plt.ylabel('$h(t)$, $h_\mathrm{BP}(t)$')
plt.show()



Wir speichern $g_\mathrm{BP}(t)$ als Audiodatei und spielen diese ab:

In [ ]:
# Save audio signal and preamble
path='audio/'
scaled = np.int16(g_bp / np.max(np.abs(g_bp)) * 32767)
write(path+'rrc.wav', np.int32(rate), scaled)

# Now we can play our audio signal!
import IPython 
IPython.display.Audio(path+'rrc.wav')